# 무신사 웹사이트에서 이미지, 코디 제목, 태그 크롤링

학습 데이터 셋을 준비하기 위해서 '무신사 웹사이트 > 코디숍' 에서 [코디 이미지 파일, 코디 제목, 코디에 해당하는 태그(list type)]을 크롤링하는 코드이다.

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 0. 연습장

In [ ]:
# 연습용 코드

import requests
from bs4 import BeautifulSoup
import os

URL = 'https://www.musinsa.com/app/styles/views/31120'

html_reshot_image = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"}).text
soup_reshot_image = BeautifulSoup(html_reshot_image, 'lxml')

reshot_image_elements = soup_reshot_image.select('a.ui-tag-list__item')

In [ ]:
[i.get_text() for i in reshot_image_elements]

['#걸리시',
 '#겨울',
 '#긴소매 티셔츠',
 '#러브참',
 '#모트',
 '#믐달',
 '#미니스커트',
 '#밀리언코르',
 '#반지',
 '#부츠',
 '#어썸니즈',
 '#토트백',
 '#톤노22',
 '#플리스/뽀글이',
 '#피그먼트',
 '#헌팅캡/베레모',
 '#플리스',
 '#뉴트럴톤',
 '#코듀로이',
 '#롱부츠',
 '#베레모',
 '#체크',
 '#화사한']

In [ ]:
imageFile

<_io.BufferedWriter name='/content/drive/MyDrive/2023 KUIAI 해커톤/detail_31122_5_500.jpg'>

## 1. 사용할 모듈 Import

In [73]:
import requests, os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## 2. 크롤링 함수 구현

In [74]:
#페이지 URL에서 리뷰 이미지, 제목, 태그 정보를 추출하는 함수 구현
'''
review_num: 무신사 홈페이지에서 추출할 코디 이미지 넘버; list type(for 문으로 반복함)
'''
def crawl_func(review_num):

    # 크롤링 사이트: 무신사 코디숍
    base_URL = 'https://www.musinsa.com/app/styles/views/'

    # 코디 이미지 넘버('image_id'), 코디 제목('title'), 태그 정보('tags_list')를 저장할 Dataframe 생성
    df = pd.DataFrame(columns = ['image_id','title','tags_list','name'])

    # 반복문을 통해 원하는 review_num에 해당하는 정보들을 크롤링
    for i in review_num:
      
      # 이미지 추출
      URL = base_URL + str(i)

      html_url = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"}).text
      soup_url = BeautifulSoup(html_url, 'lxml')

      image_elements = soup_url.select('div img')
      if not image_elements:
        continue
      image_url = 'https:' + image_elements[0].get('src')

      html_image = requests.get(image_url)

      # 이미지 파일 저장 경로
      path = '/content/drive/MyDrive/2023 KUIAI 해커톤/pic/'

      # 이미지 파일 -> .jpg 확장자로 저장
      imageFile = open(os.path.join(path, str(i)+'.jpg'), 'wb')
      chunk_size = 1000000
      for chunk in html_image.iter_content(chunk_size):
          imageFile.write(chunk)
      imageFile.close()

      # 제목 추출
      title_elements = soup_url.select('div h2')
      title = title_elements[1].get_text()

      # 태그 추출
      tags_elements = soup_url.select('a.ui-tag-list__item')
      tags = [i.get_text() for i in tags_elements]

      # 모델 이름 추출
      name_elements = soup_url.select('div > p.model_name')
      name = name_elements[0].get_text()[3:6]

      # 코디 이미지 넘버, 코디 제목, 태그 정보 -> DataFrame에 추가
      df = df.append({'image_id':i, 'title':title, 'tags_list':tags, 'name':name}, ignore_index = True)

      print(i)

      # csv로 내보내기
      df.to_csv(path + 'Musinsa_database_3.csv', encoding = 'cp949')

    return df

In [75]:
crawl_func(list(range(27846, 21162, -1)))

27846
27845
27844
27843
27842
27841
27840
27839
27838
27837
27836
27835
27834
27833
27832
27831
27830
27829
27828
27827
27826
27825
27824
27823
27822
27821
27820
27819
27818
27817
27816
27815
27814
27813
27812
27811
27810
27809
27808
27807
27806
27805
27804
27803
27802
27801
27800
27799
27798
27797
27796
27795
27794
27793
27792
27791
27790
27789
27788
27787
27786
27785
27784
27783
27782
27781
27780
27779
27778
27777
27776
27775
27774
27773
27772
27771
27770
27769
27768
27767
27766
27765
27764
27763
27762
27761
27760
27759
27758
27757
27756
27755
27754
27753
27752
27751
27750
27749
27748
27747
27746
27745
27744
27743
27742
27741
27740
27739
27738
27737
27736
27735
27734
27733
27732
27731
27730
27729
27728
27727
27726
27725
27724
27723
27722
27721
27720
27719
27718
27717
27716
27715
27714
27713
27712
27711
27710
27709
27708
27707
27706
27705
27704
27703
27702
27701
27700
27699
27698
27697
27696
27695
27694
27693
27692
27691
27690
27689
27688
27687
27686
27685
27684
27683
27682
27681
2768

KeyboardInterrupt: ignored

In [ ]:
'''
추가로 구현해야할 것

1. 사진을 폴더에 몰아서 저장할 수 있도록
2. 임의로 개수를 정했을 때 그만큼의 사진을 가져올 수 있도록 만들기 -> input값 부분 코드 수정
'''

'\n추가로 구현해야할 것\n\n1. 사진을 폴더에 몰아서 저장할 수 있도록\n2. 임의로 개수를 정했을 때 그만큼의 사진을 가져올 수 있도록 만들기 -> input값 부분 코드 수정\n'

In [72]:
df

NameError: ignored